<a href="https://colab.research.google.com/github/ykitaguchi77/Strabismus_AI_project/blob/main/ExtractList_from_imgNum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**画像番号に該当する症例をリストから抜粋するスクリプト**

In [62]:
import glob
import os
import pandas as pd
import sys
import shutil

#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pd.set_option('display.max_columns', 10)

def openCSV(csv_path):
    #df = pd.read_csv(csv_path, encoding="shift-jis")
    df = pd.read_csv(csv_path, encoding="utf-8",header=None)
    #df = pd.read_csv(csv_path, encoding="cp932")
    #df = df.drop(df.columns[[0]], axis=1) #1列目を削除する
    #print(df)
    return df

#CSVを開く （csv1:編集するファイル、csv2:参照するファイル）
csv2_path = '/content/drive/MyDrive/Deep_learning/Facephoto_LIST/DiseaseInfo_all_connected_new3.csv'
df2 = openCSV(csv2_path) #CSVを開く
df2


In [ ]:
#ファイル名のアルファベットを削除するリネーム
#path = "/content/drive/MyDrive/Deep_learning/Strabismus/dataset(3class_notcut)/train/exte/e126.jpg"
#path = "/content/drive/MyDrive/Deep_learning/Strabismus/dataset(3class_notcut)/train/exte/126.jpg"

img_path = "/content/drive/MyDrive/Deep_learning/Strabismus/Dataset_250px_20211111/exo"

orig_names = glob.glob(img_path + "/*")
for orig_name in orig_names:
    dir_name = os.path.dirname(orig_name)
    base_name = os.path.basename(orig_name)
    
    print(base_name)
    if base_name[0].isalpha() is True:
        base_name = base_name[1:] #アルファベットから始まっているものはアルファベットを削除する
    else:
        base_name = base_name

    dst_name = os.path.join(dir_name, base_name)
    os.rename(orig_name, dst_name)

In [73]:

#パスから画像名のリストを作成
img_path = "/content/drive/MyDrive/Deep_learning/Strabismus/Dataset_250px_20211111"

contlist, extelist, intelist = [], [], []

#path = "/content/drive/MyDrive/Deep_learning/Strabismus/dataset(3class_notcut)/train/exte/e126.jpg"
#このうち126の部分のみを取り出す

cont_name = glob.glob(img_path +"/cont/*")
for i in cont_name:
    try: 
        contlist.append(int(os.path.basename(os.path.splitext(i)[0])))
    except:
        pass
contlist=sorted(contlist)

exte_name = glob.glob(img_path +"/exo/*")
for i in exte_name:
    try: 
        extelist.append(int(os.path.basename(os.path.splitext(i)[0])))
    except:
        pass
extelist=sorted(extelist)
    
inte_name = glob.glob(img_path +"/eso/*")
for i in inte_name:
    try:
        intelist.append(int(os.path.basename(os.path.splitext(i)[0])))
    except:
        pass
intelist=sorted(intelist)

In [74]:
def extract(imgname_list, df_orig):
    #もとのcsvと列名を揃えたデータフレームを作成
    cols =  df_orig.columns
    df_dst = pd.DataFrame(index=[], columns=cols)

    for i in imgname_list:
        df_add = df_orig[df_orig.iloc[0:,2] ==i]
        df_dst = pd.concat([df_dst, df_add])
    return df_dst

df_cont = extract(contlist, df2)
df_exte = extract(extelist, df2)
df_inte = extract(intelist, df2)

In [75]:
import statistics
import math

def statistics_age(age_list):
    print("n=" +str(len(age_list)))
    print("max: "+str(min(age_list)))
    print("min: "+str(max(age_list)))
    print("median: "+str(statistics.median(age_list)))
    print("mean: "+str(statistics.mean(age_list)))
    print("sd: "+str(statistics.stdev(age_list)))
    
print("cont")
cont_age = df_cont.iloc[:,4].tolist()
statistics_age(cont_age)

print("")
print("exte")
exte_age = df_exte.iloc[:,4].tolist()
statistics_age(exte_age)

print("")
print("inte")
inte_age = df_inte.iloc[:,4].tolist()
statistics_age(inte_age)

print("")
print("total")
total_age = cont_age + exte_age + inte_age
statistics_age(total_age)

cont
n=366
max: 0
min: 83
median: 10.0
mean: 17.959016393442624
sd: 20.37137296036135

exte
n=334
max: 0
min: 82
median: 7.0
mean: 16.856287425149702
sd: 20.81428691858433

inte
n=294
max: 0
min: 86
median: 5.0
mean: 17.755102040816325
sd: 24.12357034593524

total
n=994
max: 0
min: 86
median: 7.5
mean: 17.528169014084508
sd: 21.674227000752918


In [76]:
#年齢の母集団に有意差があるかどうかの解析（Kruskal-Wallis)
from scipy.stats.mstats import kruskal
from scipy.stats import ttest_ind
print("Kruskal-Wallis")
print(kruskal(inte_age, exte_age, inte_age))

print("t-test_ind")
print("exo vs eso: "+str(ttest_ind(exte_age, inte_age, equal_var = False)))
print("exo vs cont: "+str(ttest_ind(exte_age, cont_age, equal_var = False)))
print("cont vs eso: "+str(ttest_ind(cont_age, inte_age, equal_var = False)))

Kruskal-Wallis
KruskalResult(statistic=5.7522466839128334, pvalue=0.05635280046961159)
t-test_ind
exo vs eso: Ttest_indResult(statistic=-0.4965508504086252, pvalue=0.6196930945735439)
exo vs cont: Ttest_indResult(statistic=-0.7072598729760249, pvalue=0.4796440744617124)
cont vs eso: Ttest_indResult(statistic=0.11556846948727542, pvalue=0.9080349919099393)


In [78]:
a = [6.3, 5.4, 7.7, 5.2, 5.0]
b = [6.9, 7.0, 6.1, 7.9]
c = [7.2, 6.9, 6.1, 6.5]
kruskal(a, b, c)

KruskalResult(statistic=2.814364640883984, pvalue=0.2448321707681035)

#**CSVから該当する文字列のある行を抜き出すサンプルコード**

In [53]:
#df2の0列目から"2695701"を見つけて、その行を抜き出す
#さらにその行の2列目の内容を出力する

img_num = df2[df2.iloc[0:,2] == 100].iloc[0,0]
print(img_num)

10065813.0


#**CSVとして保存**

In [ ]:
csv_path = '***********.csv'
csv = df1.to_csv(csv_path, encoding='utf_8_sig')
